## Options Backtesting Framework

This is a simple backtesting framework for options. Given end of day data for both the capital and derivatives markets, we would be backtesting for different strategies.

> All options are opened at the start of the period and closed by the end of the period or by a stop loss if provided

### Implementation

1. Load and transform necessary data
2. Create a list of option contracts to be entered into each day
3. Create a list of signals on, whether to enter into the contract, for each day
4. Extract the necessary data for the options contracts that would be executed
5. Analyze Profit and loss and other metrices


In [9]:
# Load necessary libraries

import pandas as pd
import numpy as np
from fastbt.rapid import backtest

## Utility Functions

In [24]:
def itm(price, opt='C', step=100):
    """
    Get the strike price of the in the money option
    price
        spot price
    opt
        option type - C for Call and P for Put
    step
        multiple of option strike price
    """
    if opt == 'C':
        return int(price/step)*step
    elif opt == 'P':
        return (int(price/step)+1)*step

10225

Data Preparation
-----------------
Load the raw data and do some transformations. All raw options data is loaded from the bhav copy of the NSE website. Index data is from [niftyindices.com](https://www.niftyindices.com/).

In [10]:
nifty = pd.read_csv('/home/machine/Downloads/NIFTY 50_Data.csv',
                   parse_dates=['Date'])